In [18]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy, # uses Embeddings
    context_precision, # uses Embeddings
    context_recall, # uses ground_truth dataset (only LLM used as a jugdge)
)
from datasets import Dataset

from ragas.llms import LangchainLLMWrapper
from langchain_community.chat_models import ChatOllama
from ragas.run_config import RunConfig
import pandas as pd

In [ ]:


def extract_queries() -> pd.DataFrame:
    url_source: str = (
        "https://docs.google.com/spreadsheets/d/e/2PACX-1vR1hUlRhTJQgNzSbTyRtDNh1mCrbfy0iUm6oiHK7oHb_iQQ5t7XCB_xyUCwoZ2fdg/pub?output=xlsx"
    )
    queries = pd.read_excel(url_source, sheet_name="queries")
    return queries


def transform_queries(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    return df


def load_queries() -> pd.DataFrame:
    df = extract_queries()
    df = transform_queries(df)
    return df

In [3]:
queries = load_queries()
queries

,query_id,query,answer
0,1,Paano magapply ng Japan Visa?,Complete all the requirements based on the pur...
1,2,Saan pwede magprocess ng Japan Visa?,Pwede mag process ng Japan Visa sa Attic Tours...
2,3,Magkano ang processing fees ng Japan Visa?,Ang processing fee ng Japan Visa sa Attic Tour...
3,4,Ano ang mga requirements pag mag tourist sa Ja...,"Kailangan ng Philippine Passport original, Col..."
4,5,Ano ang mga types ng Japan Visa?,"Ang mga types ng Japan Visa ay tourist, visiti..."


In [4]:
from rag_chatbot import RAG_Chatbot

chatbot = RAG_Chatbot()
chatbot.prepare_docs()

/Users/krimssmirk/Desktop/rag/.venv/lib/python3.11/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.31.1 at v1/aggregate.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/Users/krimssmirk/Desktop/rag/.venv/lib/python3.11/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.31.1 at v1/base.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/Users/krimssmirk/Desktop/rag/.venv/lib/python3.11/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.31.1 at v1/base_search.proto. Please update the gencode to avoid compatibility violations in the next ru

Requirements has been removed!
New Collection Requirements created!
📄 Processing: MULTIPLE-ENTRY VISA FOR TEMPORARY VISITOR.pdf


/Users/krimssmirk/Desktop/rag/.venv/lib/python3.11/site-packages/torch/nn/modules/module.py:1520: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Inserted object chunk_id=MULTIPLE-ENTRY VISA FOR TEMPORARY VISITOR.pdf_chunk_0 title=MULTIPLE-ENTRY VISA FOR TEMPORARY VISITOR
📄 Processing: FREQUENTLY ASKED QUESTIONS.pdf
Inserted object chunk_id=FREQUENTLY ASKED QUESTIONS.pdf_chunk_0 title=Frequently Asked Questions (Visa
📄 Processing: ATTIC TOURS.pdf
Inserted object chunk_id=ATTIC TOURS.pdf_chunk_0 title=ATTIC TOURS
📄 Processing: VISITING RELATIVES.pdf
Inserted object chunk_id=VISITING RELATIVES.pdf_chunk_0 title=VISITING RELATIVES
📄 Processing: VISITING US MILITARY PERSONNEL.pdf
Inserted object chunk_id=VISITING US MILITARY PERSONNEL.pdf_chunk_0 title=VISITING US MIILITALY PERSONNEL
📄 Processing: MULTIPLE-ENTRY TEMPORARY VISITOR VISA (PHILIPPINE NATIONALS WITH CONSIDERABLE FINANCIAL CAPACITY).pdf
Inserted object chunk_id=MULTIPLE-ENTRY TEMPORARY VISITOR VISA (PHILIPPINE NATIONALS WITH CONSIDERABLE FINANCIAL CAPACITY).pdf_chunk_0 title=MULTIPLE-ENTRY TEMPORARY VISITOR VISA (PHILIPPINE NATIONALS WITH
📄 Processing: TRANSIT.pdf
Inserte

In [7]:
def test_rag_chatbot() -> tuple[list[list[str]], list[str]]:
    retrieved_texts = []
    model_answers = []
    for query in queries['query']:
        relevant_docs = chatbot._retrieved_relevant_docs(query, top_k=5)
        messages = chatbot._get_messages(query=query, context=relevant_docs)
        model_answer = ""
        for response_text in chatbot._generate_response(messages):
            model_answer += response_text
        retrieved_texts.append(relevant_docs)
        model_answers.append(model_answer)
    return retrieved_texts, model_answers

In [8]:
retrieved_texts, model_answers = test_rag_chatbot()

/Users/krimssmirk/Desktop/rag/.venv/lib/python3.11/site-packages/torch/nn/modules/module.py:1520: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [28]:
eval_llm = ChatOllama(model="llama3:8b-instruct", temperature=0.0)
ragas_llm = LangchainLLMWrapper(eval_llm)

In [29]:
data = {
    "question": queries["query"].to_list(),
    "answer": model_answers,
    "contexts": retrieved_texts,
    "ground_truth": queries["answer"].to_list(),
}
ds = Dataset.from_dict(data)

In [30]:
metrics = [
    faithfulness
]

In [ ]:
# Define a custom timeout, for example, 300 seconds (5 minutes)
# Adjust this value based on your hardware and dataset size
run_config = RunConfig(timeout=7200)

result = evaluate(
    ds,
    metrics=metrics,
    llm=ragas_llm,
    run_config=run_config
)

Evaluating:   0%|          | 0/5 [00:00<?, ?it/s]Prompt fix_output_format failed to parse output: The output parser failed to parse the output including retries.
Prompt fix_output_format failed to parse output: The output parser failed to parse the output including retries.
Prompt fix_output_format failed to parse output: The output parser failed to parse the output including retries.
Prompt statement_generator_prompt failed to parse output: The output parser failed to parse the output including retries.
Exception raised in Job[2]: RagasOutputParserException(The output parser failed to parse the output including retries.)
Evaluating:  20%|██        | 1/5 [35:13<2:20:55, 2113.81s/it]Prompt fix_output_format failed to parse output: The output parser failed to parse the output including retries.
Prompt fix_output_format failed to parse output: The output parser failed to parse the output including retries.
Prompt fix_output_format failed to parse output: The output parser failed to parse 

In [31]:
result

{'faithfulness': nan}